In [1]:
import os
import time
import numpy as np
import tensorflow as tf

In [2]:
#path_to_file= tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
path_to_file ="./les1.txt"
text=open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f'Length of text:{len(text)} characters')

Length of text:10164 characters


In [3]:
print(text[:250])

Лесь Подерв’янський.
Павлік Морозов.
Епічна трагедія.
Дійові особи
Павлiк Морозов, атлетичний юнак, нордична краса i гiтлерюгендiвська зачiска; одягнут просто i зi смаком: в бiлу сорочку i короткi шкiрянi штанцi; з рiзних бокiв Павлiк Морозов пер


In [4]:
vocab=sorted(set(text))
print(f'{len(vocab)} unique characters')

85 unique characters


In [5]:
chars = tf.strings.unicode_split(path_to_file, input_encoding='UTF-8')


In [8]:
ids_from_chars=tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
chars_from_ids =tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)
chars

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b'.', b'[UNK]', b'l', b'e', b'[UNK]', b'[UNK]', b'.', b'[UNK]',
       b'[UNK]', b'[UNK]'], dtype=object)>

In [7]:
ids = ids_from_chars(chars)
ids

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 9,  0, 17, 15,  0,  0,  9,  0,  0,  0])>

In [9]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [10]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(10164,), dtype=int64, numpy=array([34, 55, 67, ..., 69, 85,  9])>

In [11]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [12]:
for ids in ids_dataset.take(100):
    print(chars_from_ids(ids).numpy().decode('utf-8'))
#for ids in ids_dataset.take(40):
    #print(chars_from_ids(ids).numpy().decode('utf-8'))

seq_length = 100

Л
е
с
ь
 
П
о
д
е
р
в
’
я
н
с
ь
к
и
й
.



П
а
в
л
і
к
 
М
о
р
о
з
о
в
.



Е
п
і
ч
н
а
 
т
р
а
г
е
д
і
я
.



Д
і
й
о
в
і
 
о
с
о
б
и



П
а
в
л
i
к
 
М
о
р
о
з
о
в
,
 
а
т
л
е
т
и
ч
н
и
й
 
ю
н


In [13]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

#for seq in sequences.take(1):
  #print(chars_from_ids(seq))

In [14]:
#len(sequences)
#for seq in sequences.take(2):
  #print(text_from_ids(seq).numpy())

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [15]:
#split_input_target(list("Tensorflow"))
dataset = sequences.map(split_input_target)
dataset

<_MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>

In [16]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\xd0\x9b\xd0\xb5\xd1\x81\xd1\x8c \xd0\x9f\xd0\xbe\xd0\xb4\xd0\xb5\xd1\x80\xd0\xb2\xe2\x80\x99\xd1\x8f\xd0\xbd\xd1\x81\xd1\x8c\xd0\xba\xd0\xb8\xd0\xb9.\r\n\xd0\x9f\xd0\xb0\xd0\xb2\xd0\xbb\xd1\x96\xd0\xba \xd0\x9c\xd0\xbe\xd1\x80\xd0\xbe\xd0\xb7\xd0\xbe\xd0\xb2.\r\n\xd0\x95\xd0\xbf\xd1\x96\xd1\x87\xd0\xbd\xd0\xb0 \xd1\x82\xd1\x80\xd0\xb0\xd0\xb3\xd0\xb5\xd0\xb4\xd1\x96\xd1\x8f.\r\n\xd0\x94\xd1\x96\xd0\xb9\xd0\xbe\xd0\xb2\xd1\x96 \xd0\xbe\xd1\x81\xd0\xbe\xd0\xb1\xd0\xb8\r\n\xd0\x9f\xd0\xb0\xd0\xb2\xd0\xbbi\xd0\xba \xd0\x9c\xd0\xbe\xd1\x80\xd0\xbe\xd0\xb7\xd0\xbe\xd0\xb2, \xd0\xb0\xd1\x82\xd0\xbb\xd0\xb5\xd1\x82\xd0\xb8\xd1\x87\xd0\xbd\xd0\xb8\xd0\xb9 \xd1\x8e\xd0\xbd'
Target: b'\xd0\xb5\xd1\x81\xd1\x8c \xd0\x9f\xd0\xbe\xd0\xb4\xd0\xb5\xd1\x80\xd0\xb2\xe2\x80\x99\xd1\x8f\xd0\xbd\xd1\x81\xd1\x8c\xd0\xba\xd0\xb8\xd0\xb9.\r\n\xd0\x9f\xd0\xb0\xd0\xb2\xd0\xbb\xd1\x96\xd0\xba \xd0\x9c\xd0\xbe\xd1\x80\xd0\xbe\xd0\xb7\xd0\xbe\xd0\xb2.\r\n\xd0\x95\xd0\xbf\xd1\x96\xd1\x87\xd0\xbd\xd0\xb0 \

In [17]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [18]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
print(vocab_size)
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024


85


In [19]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, return_sequences=True)))
model.add(tf.keras.layers.Dense(vocab_size, activation="softmax"))



In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 85) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         21760     
                                                                 
 bidirectional (Bidirectiona  (None, None, 2048)       10493952  
 l)                                                              
                                                                 
 dense (Dense)               (None, None, 85)          174165    
                                                                 
Total params: 10,689,877
Trainable params: 10,689,877
Non-trainable params: 0
_________________________________________________________________


In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([66,  3, 42, 77, 42, 78, 24, 25, 53, 67, 84, 66, 51, 24, 78, 81,  5,
        5, 14, 58, 38, 29, 48, 12, 75, 40, 39, 45, 59,  4, 77, 63, 17, 84,
       69, 49, 39,  9, 76, 28, 74,  1, 20, 27, 63, 74,  2, 39, 37, 83, 11,
       10,  4,  9, 77,  3, 17, 42, 18, 64, 60, 21, 52, 24, 61, 48, 42, 65,
       22, 48, 46, 28, 72, 36, 55, 84, 65, 53, 77, 18, 55, 81, 80, 63, 67,
       77,  7, 31, 65, 39, 17, 17, 40, 47, 34, 30, 11, 34, 35, 43])

In [23]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy().decode('utf-8'))
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy().decode('utf-8'))

Input:
 iзку, якiй зовсiм недавно признавався в любвi. Берiзка ламається.

Учитель атеїзму (до дiтей).
Дє

Next Char Predictions:
 р УюУяАБгс“рбАяї((bиПЕЩ?щСРЦй!юнl“уЯР.ьДш
РО’;:!.ю lУmокrвАлЩУпІЩЧДцНе“пгюmеїінсю,ЗпРllСШЛЖ;ЛМФ


# **Train the model**

In [24]:
adam=tf.keras.optimizers.Adam() #(learning_rate=0.01)
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(loss=loss, optimizer=adam) # metrics=['accuracy'])
#history=model.fit(xs, ys, epochs =100, verbose =1)

In [25]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 85)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.4417105, shape=(), dtype=float32)


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


In [26]:
tf.exp(example_batch_mean_loss).numpy()

84.920074

In [27]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [28]:
EPOCHS = 100

In [29]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
1/1 [==============================] - 26s 26s/step - loss: 4.4417
Epoch 2/100
1/1 [==============================] - 13s 13s/step - loss: 4.3958
Epoch 3/100
1/1 [==============================] - 13s 13s/step - loss: 4.1059
Epoch 4/100
1/1 [==============================] - 12s 12s/step - loss: 7.1727
Epoch 5/100
1/1 [==============================] - 14s 14s/step - loss: 3.9633
Epoch 6/100
1/1 [==============================] - 13s 13s/step - loss: 3.9037
Epoch 7/100
1/1 [==============================] - 13s 13s/step - loss: 4.0397
Epoch 8/100
1/1 [==============================] - 13s 13s/step - loss: 4.0378
Epoch 9/100
1/1 [==============================] - 12s 12s/step - loss: 3.9796
Epoch 10/100
1/1 [==============================] - 13s 13s/step - loss: 3.9265
Epoch 11/100
1/1 [==============================] - 12s 12s/step - loss: 3.8167
Epoch 12/100
1/1 [==============================] - 14s 14s/step - loss: 3.7176
Epoch 13/100
1/1 [==============================]

In [30]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [31]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [32]:
#start = time.time()
#states = None
#next_char = tf.constant(['Павлiк Морозов:'])
#result = [next_char]

#for n in range(2000):
  #next_char, states = one_step_model.generate_one_step(next_char, states=states)
  #result.append(next_char)

#result = tf.strings.join(result)
#end = time.time()
#print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
#print('\nRun time:', end - start)

In [33]:
model.save("./les1.h5")

In [1]:
!pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!tensorflowjs_converter --input_format=keras "./les1.h5" ./

2023-06-04 13:54:12.010915: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 13:54:15.639034: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
